In [21]:
import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json


import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as wd2

In [22]:
colorway = ['#702632', '#A4B494', '#495867', '#912F40', "#81909E", "#F4442E", "#DB7C26", "#BB9BB0"]
pio.templates["cardinal"] = go.layout.Template(
    layout=go.Layout(
        colorway=colorway
    ))
# combine templates 
pio.templates.default = 'plotly_white+cardinal'

## real data 

In [23]:
a00, a01 = h.import_desired_data("A", "15T")
# print(a00["Window Open"].unique(), a01["Window Open"].unique(), a00.columns)

In [31]:
w2 = wd2.Window_Detect2(a01)
w2.analyze_window_change(wd2.make_sin_smooth)

w3 = wd2.Window_Detect2(a01)
w3.analyze_window_change(wd2.make_ewm_smooth)

In [71]:


def make_dual_plot(obj1, obj2): 
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True)

    s2, s3 = [[obj.window_norm, obj.temp_norm, obj.smooth_series, obj.dif, obj.deriv, obj.deriv2] for obj in [obj1, obj2]]

    names = ["Window", "Observed Temp", "Smoothed", "Difference", "Deriv1", "Deriv2"]
    
    opacities = [0.4] + [1]*(len(s2)-1)

    for ix, name, ser in zip(range(len(names)), names, s2):
        fig.add_trace(go.Scatter(x=w2.time, y=ser, name=name, mode='lines', line_color=colorway[ix], opacity=opacities[ix],legendgroup=name,), row = 1, col = 1)

    for ix, name, ser in zip(range(len(names)), names, s3):
        fig.add_trace(go.Scatter(x=w2.time, y=ser, name=name, mode='lines', line_color=colorway[ix], opacity=opacities[ix],legendgroup=name, showlegend=False), row = 1, col = 2)

    return fig, names

In [72]:
def update_dual_plot(fig, names, show_arr, ):
    for name in names: 
        if name in show_arr:
            fig.update_traces(visible=True, selector=dict(name=name))
        else:
            fig.update_traces(visible="legendonly", selector=dict(name=name))
    return fig
   


In [73]:
fig, names = make_dual_plot(w2, w3)
update_dual_plot(fig,  names, ["Window", "Observed Temp", "Smoothed"],)

In [75]:
update_dual_plot(fig, names, ["Deriv1", "Deriv2"])

In [61]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)

s2, s3 = [[obj.window_norm, obj.temp_norm, obj.smooth_series, obj.deriv] for obj in [w2, w3]]
names = ["Window", "Observed", "Smoothed", "Deriv1"]
opacities = [0.4, 1, 1,1]

for ix, name, ser in zip(range(len(names)), names, s2):
    fig.add_trace(go.Scatter(x=w2.time, y=ser, name=name, mode='lines', line_color=colorway[ix], opacity=opacities[ix],legendgroup=name,), row = 1, col = 1)

for ix, name, ser in zip(range(len(names)), names, s3):
    fig.add_trace(go.Scatter(x=w2.time, y=ser, name=name, mode='lines', line_color=colorway[ix], opacity=opacities[ix],legendgroup=name, showlegend=False), row = 1, col = 2)

fig.update_traces(visible="legendonly", selector=dict(name="Deriv1"))


fig.show()


In [62]:
fig.update_traces(visible="legendonly", selector=dict(name="Observed"))
fig.update_traces(visible="legendonly", selector=dict(name="Smoothed"))
fig.update_traces(visible=True, selector=dict(name="Deriv1"))

## simulation data 

In [25]:
# sim_data_path = "../../data/energy_model/230307.pkl"
# sim_data = pd.read_pickle(sim_data_path)

# # update year to match ! NOTE: this has not been calibrated, so based on EPW data from 2017!
# delta = pd.Timestamp('2022-01-01') - pd.Timestamp('2017-01-01')
# sim_data["DateTime"] += delta 

# # trim so match real data 
# start = a01["DateTime"][0]
# end = a01["DateTime"].iloc[-1]
# mask = (sim_data['DateTime'] >= start) & (sim_data['DateTime'] <= end)
# sim_data = sim_data.loc[mask]

# sim_data.to_pickle("../../data/energy_model/230307_match.pkl")

In [27]:
sim_data_path = "../../data/energy_model/230307_match.pkl"
sim_data = pd.read_pickle(sim_data_path)

# smoothing on different time series than the observation 
sim_smooth = wd2.make_stl_smooth(series=sim_data["Always Closed"], time=None, model="additive")

# rename label in simulation dataframe to match class
sim_data.rename(columns={"Varied": "Temp C"}, inplace=True)

In [29]:
s1 = wd2.Window_Detect2(sim_data)
s1.analyze_window_change(smooth_fx=None, sim_smooth=sim_smooth)
